In [1]:
import networkx as nx
from area import area
from locus.utils.cell_utils import cell_center
from locus.utils.cell_utils import CellState, cell_bounds
from locus.utils.paths import PROCESSED_DATA_DIR

In [2]:
G = nx.read_gml(PROCESSED_DATA_DIR / "LDoGI/quadtrees/qt_min10_max1000_df10pct.gml")
active_cells = [node for node in list(G.nodes) if G.nodes[node]["state"] == CellState.ACTIVE.value]

In [3]:
active_cells[1352]

'0133133133102'

In [4]:
cb = cell_bounds(active_cells[1352])
cc = cell_center(active_cells[1352])
cb = {"south": cb[0], "north": cb[1], "west": cb[2], "east": cb[3]}
print(cb)
print(cc)

{'south': 51.4599609375, 'north': 51.48193359375, 'west': -0.17578125, 'east': -0.1318359375}
(51.470947265625, -0.15380859375)


In [5]:
obj = {
    "type": "Polygon",
    "coordinates": [
        [
            [cb["west"], cb["south"]],
            [cb["west"], cb["north"]],
            [cb["east"], cb["north"]],
            [cb["east"], cb["south"]],
        ]
    ],
}

In [6]:
area(obj)

7453561.007267797

In [7]:
from geopy import distance

In [8]:
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth's surface given their latitude and longitude
    in decimal degrees.
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth radius in kilometers
    return distance

def distance_to_square(lat, lon, south_lat, north_lat, west_lon, east_lon):
    """
    Calculate the distance between a latitude-longitude coordinate 
    and a square bounded by south latitude, north latitude, 
    west longitude, and east longitude.
    """
    # Check if the point is within the latitude and longitude bounds of the square
    if south_lat <= lat <= north_lat and west_lon <= lon <= east_lon:
        return 0.0  # The point is inside the square

    # Calculate the distance to each edge of the square
    distances = [
        haversine(lat, lon, south_lat, lon),   # Distance to south edge
        haversine(lat, lon, north_lat, lon),   # Distance to north edge
        haversine(lat, lon, lat, west_lon),    # Distance to west edge
        haversine(lat, lon, lat, east_lon)     # Distance to east edge
    ]

    # Return the minimum distance to any edge of the square
    return min(distances)

In [9]:
def distance_to_square2(lat, lon, south_lat, north_lat, west_lon, east_lon):
    """
    Calculate the distance between a latitude-longitude coordinate
    and a square bounded by south latitude, north latitude,
    west longitude, and east longitude.
    """
    # Check if the point is within the latitude and longitude bounds of the square
    if south_lat <= lat <= north_lat and west_lon <= lon <= east_lon:
        return 0.0  # The point is inside the square

    def clamp(n, low, high):
        return max(low, min(n, high))

    # Calculate the distance to each edge of the square
    distances = [
        distance.great_circle((lat, lon), (south_lat, clamp(lon, west_lon, east_lon))),  # Distance to south edge
        distance.great_circle((lat, lon), (north_lat, clamp(lon, west_lon, east_lon))),  # Distance to north edge
        distance.great_circle((lat, lon), (clamp(lat, south_lat, north_lat), west_lon)),  # Distance to west edge
        distance.great_circle((lat, lon), (clamp(lat, south_lat, north_lat), east_lon)),  # Distance to east edge
    ]

    # Return the minimum distance to any edge of the square
    return min(distances)

In [10]:
# Example usage:
lat = 40.7128  # Latitude of the point
lon = -76.0060  # Longitude of the point
south_lat = 40.0  # South latitude of the square
north_lat = 41.0  # North latitude of the square
west_lon = -75.0  # West longitude of the square
east_lon = -73.0  # East longitude of the square

d = distance_to_square(lat, lon, south_lat, north_lat, west_lon, east_lon)
print("Distance to square:", d, "km")


d2 = distance_to_square2(lat, lon, south_lat, north_lat, west_lon, east_lon)
print("Distance to square:", d2, "km")

Distance to square: 31.935182932317066 km
Distance to square: 84.78985423622076 km km


In [11]:
haversine(lat, lon, (south_lat+north_lat)/2, (west_lon+east_lon)/2)

170.98580126944051

In [12]:
distance.great_circle((lat, lon), ((south_lat+north_lat)/2, (west_lon+east_lon)/2)).km

170.9860428127165

In [13]:
distance.distance((lat, lon), ((south_lat+north_lat)/2, (west_lon+east_lon)/2)).km

171.40790026529942

In [14]:
distance_to_square2(lat, lon, south_lat, north_lat, west_lon, east_lon)

Distance(84.78985423622076)

In [17]:
d2.km

84.78985423622076